In [20]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
import sys
import time
import warnings
import matplotlib as mpl
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
torch.cuda.is_available()

True

In [21]:
train_all = pd.read_csv('train.csv')
test_all = pd.read_csv('test.csv')

datasets = [train_all, test_all]

In [22]:
train_all.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [23]:
train_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [24]:
# for dataset in datasets:
#   dataset.drop(["Alley", "MiscFeature", "PoolQC", "Fence"], axis=1, inplace = True)
#   for col in dataset.columns:
#     if dataset[col].isnull().sum() > 1000:
#       print(col, dataset[col].isnull().sum())


In [25]:
corrmat = train_all.corr()
corrmat["SalePrice"].sort_values(ascending=False)



SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePr

In [26]:
train_all = train_all.loc[:, ["SalePrice", "OverallQual", "GrLivArea", "GarageCars", "GarageArea",
                              "TotalBsmtSF", "1stFlrSF", "FullBath", "TotRmsAbvGrd", "YearBuilt", "YearRemodAdd"]]
test_all = test_all.loc[:, ["OverallQual", "GrLivArea", "GarageCars", "GarageArea",
                            "TotalBsmtSF", "1stFlrSF", "FullBath", "TotRmsAbvGrd", "YearBuilt", "YearRemodAdd"]]

train_all.describe()
train_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   SalePrice     1460 non-null   int64
 1   OverallQual   1460 non-null   int64
 2   GrLivArea     1460 non-null   int64
 3   GarageCars    1460 non-null   int64
 4   GarageArea    1460 non-null   int64
 5   TotalBsmtSF   1460 non-null   int64
 6   1stFlrSF      1460 non-null   int64
 7   FullBath      1460 non-null   int64
 8   TotRmsAbvGrd  1460 non-null   int64
 9   YearBuilt     1460 non-null   int64
 10  YearRemodAdd  1460 non-null   int64
dtypes: int64(11)
memory usage: 125.6 KB


In [27]:
for col in test_all.columns:
  test_all[col].fillna(test_all[col].mean(), inplace=True)

In [28]:
train, valid = train_test_split(
    train_all, test_size=0.1, random_state=42, shuffle=True)

train_X, train_y = train.drop('SalePrice', axis=1), train['SalePrice']
valid_X, valid_y = valid.drop('SalePrice', axis=1), valid['SalePrice']
# num_pip = Pipeline([
#     ('imputer', SimpleImputer(strategy="median")),
#     ('std_scaler', StandardScaler()),
# ])

# train_X = num_pip.fit_transform(train_X)
train_X

,OverallQual,GrLivArea,GarageCars,GarageArea,TotalBsmtSF,1stFlrSF,FullBath,TotRmsAbvGrd,YearBuilt,YearRemodAdd
907,7,2057,1,180,1017,1020,1,6,1936,1987
782,7,1430,2,605,1413,1430,2,6,2001,2002
952,5,768,1,396,768,768,1,5,1972,2003
620,3,864,0,0,864,864,1,5,1914,1950
669,4,1180,1,252,700,1180,1,5,1922,1950
...,...,...,...,...,...,...,...,...,...,...
1095,6,1314,2,440,1314,1314,2,6,2006,2006
1130,4,1981,2,576,1122,1328,2,7,1928,1950
1294,5,864,2,572,864,864,1,5,1955,1990
860,7,1426,1,216,912,912,1,7,1918,1998


In [29]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)
model = keras.Sequential([
    keras.layers.Dense(16, activation='elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(32, activation='elu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(64, activation='elu'),
    # keras.layers.BatchNormalization(),
    # keras.layers.Dropout(0.05),
    # keras.layers.Dense(128, activation='elu'),
    # keras.layers.BatchNormalization(),
    # keras.layers.Dropout(0.05),
    keras.layers.Dense(1)
])

In [30]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='mean_squared_error')

In [31]:
model.fit(train_X, train_y, epochs=20, validation_data=(valid_X, valid_y))

Epoch 1/20
42/42 [==============================] - 1s 7ms/step - loss: 38645293056.0000 - val_loss: 42572054528.0000
Epoch 2/20
42/42 [==============================] - 0s 4ms/step - loss: 38642446336.0000 - val_loss: 42557693952.0000
Epoch 3/20
42/42 [==============================] - 0s 4ms/step - loss: 38635311104.0000 - val_loss: 42529730560.0000
Epoch 4/20
42/42 [==============================] - 0s 4ms/step - loss: 38622408704.0000 - val_loss: 42487525376.0000
Epoch 5/20
42/42 [==============================] - 0s 4ms/step - loss: 38602055680.0000 - val_loss: 42435960832.0000
Epoch 6/20
42/42 [==============================] - 0s 4ms/step - loss: 38571642880.0000 - val_loss: 42372067328.0000
Epoch 7/20
42/42 [==============================] - 0s 4ms/step - loss: 38530945024.0000 - val_loss: 42286153728.0000
Epoch 8/20
42/42 [==============================] - 0s 4ms/step - loss: 38477713408.0000 - val_loss: 42191507456.0000
Epoch 9/20
42/42 [==============================] - 0s 4

In [32]:
y_pred = model.predict(test_all)
y_pred

array([[ 2242.499 ],
       [ 9793.799 ],
       [13454.887 ],
       ...,
       [10918.637 ],
       [  138.7028],
       [16001.619 ]], dtype=float32)

In [33]:
prediction = pd.DataFrame(y_pred, columns=['SalePrice'])

prediction["Id"] = pd.read_csv('./test.csv')["Id"]

prediction.to_csv("./submit.csv", index=False, header=True)


# .to_csv('submit.csv')
# prediction

In [34]:
test_pred = model.predict(train_X[:5])
test_pred

array([[13180.268  ],
       [14607.22   ],
       [  164.9984 ],
       [  254.61414],
       [ 2238.431  ]], dtype=float32)

In [35]:
train_y[:5]

907    250000
782    187100
952    133900
620     67000
669    137500
Name: SalePrice, dtype: int64